Coursera Capstone - Battle of the Neighborhoods

In [ ]:
Data Sources :
#Venue Data
Foursquare.com 
#Location and Borough Data
Geonames : Cambridge Borough Data Set and GPS data Downloaded from Geonames (in the UK_full zip file)
http://download.geonames.org/export/zip/


